In [2]:
%%python

# Tree class to build decision tree
class Tree:
    """Abstract base class for trees"""
    
    class Position:
        """Nested class for location of an element"""
        def element(self):
            raise NotImplementedError('must be impmlemented in a sub-class')
            
        def __eq__(self,other):
            raise NotImplementedError('must be impmlemented in a sub-class')

        def __ne__(self,other):
            raise NotImplementedError('must be impmlemented in a sub-class')
            
    def root(self):
        raise NotImplementedError('must be impmlemented in a sub-class')
    def parent(self,p):
        raise NotImplementedError('must be impmlemented in a sub-class')
    def num_children(self,p):
        raise NotImplementedError('must be impmlemented in a sub-class')
    def children(self,p):
        raise NotImplementedError('must be impmlemented in a sub-class')
    def __len__(self):
        raise NotImplementedError('must be impmlemented in a sub-class')

    def is_root(self,p):
        """return true if position p is the root"""
        return self.root()==p
    def is_leaf(self,p):
        """return true if position p is a leaf"""
        return self.num_children(p)==0
    def is_empty(self):
        """return true if the tree is empty"""
        return len(self)==0
    
class LinkedTree(Tree):
    """Linked representation of a general tree structure"""
    
    class _Node:
        """Nested class for storing nodes of binary trees"""
        __slots__ = ('_element','_parent','_children')
        def __init__(self, element, parent = None, children = []):
            self._element = element
            self._parent = parent
            self._children = children

    class Position(Tree.Position):
        """Nested class for location of an element"""
        
        def __init__(self, container, node):
            self._container = container
            self._node = node
        
        def element(self):
            """returns the element at the position"""
            return self._node._element
        
        def __eq__(self,other):
            """Return True if other position represents the same location"""
            return type(other) is type(self) and other._node is self._node


    def _validate(self,p):
        """Return associated node, if position is valid"""
        if not isinstance(p,self.Position):
            raise TypeError('p must be a proper Position type')
        if p._container is not self:
            raise ValueError('p does not belong to this container')
        if p._node._parent is p._node:
            raise ValueError('p is no longer valid')
        return p._node
    
    def _make_position(self,node):
        """Return Position instance for given node"""
        return self.Position(self,node) if node is not None else None



    def __init__(self):
        """Constructor for Binary tree"""
        self._root = None
        self._size = 0
    
    def __len__(self):
        """Returns the size of the tree"""
        return self._size
    
    def root(self):
        """Return the root Position of the tree (or None if p is root)."""
        return self._make_position(self._root)
    
    def parent(self,p):
        """Return the position of p's parent"""
        node = self._validate(p)
        return self._make_position(node._parent)

    def num_children(self,p):
        """Return the number of children of position p"""
        node = self._validate(p)        
        return len(node._children)
    
    def _add_root(self,e):
        """Create a root for the graph"""
        if self._root is not None: raise ValueError('Root Exists')
        self._size = 1
        self._root = self._Node(e,None,[])
        return self._make_position(self._root)

    def siblings(self,p):
        """Returns a list of position of the siblings of p"""
        node = self._validate(p)
        
        # l is a list of positions
        l = []
        # parent is nodes p's parent
        parent = node._parent
        # siblings is a list of the nodes p's siblings
        sibs = parent._children
        # loop over the nodes of p's children
        for child in sibs:
            # create a position for the child of p
            q = self._make_position(child)
            if q == p:
                pass
            else:
                #append the position to the list
                l.append(q)
            
        return l
        
    def children(self,p):
        """Return a list of the position of the children of p"""
        # validate the position coming in
        node = self._validate(p)
        # children is a list of the nodes p's children
        children = node._children
        
        return list(map(self._make_position,children))
    
    def _add_child(self,p,e):
        """Add a new child to the node at position p with element e"""
        # validate the parent position p
        node_p = self._validate(p)

        # Create a node for the new child
        node_c = self._Node(e,node_p,[])

        # append child to the list
        node_p._children.append(node_c)
        
        # increment the size
        self._size += 1
        
        # return the position of the newest child node
        return self._make_position(node_c)
    
    def _replace(self,p,e):
        """replace the element at position p with e"""
        node = self._validate(p)
        old = node._element
        node._element = e
        return old

    def _delete(self,p):
        """Delete the node at position p if it is a leaf"""
        node_p= self._validate(p)
        # check if p is a leaf
        if self.num_children(p)>0: raise ValueError('p is not a leaf')
        # parent is the node for the parent of p
        parent = node_p._parent

        kids = self.children(self._make_position(parent))
        # Loop over the children of p        
        for i in range(len(kids)):
            # check if we found the node to delete
            if kids[i]==p:
                break
        # remove the node from the parent node's children list
        parent._children.pop(i)
        # Decrement the size of the tree        
        self._size -= 1

        return node_p._element
    
    def _attach(self,p,t1):
        """Attach trees t1 as a subtrees of external p"""
        # validate position
        node = self._validate(p)
        # Check the position is a leaf
        if not self.is_leaf(p): raise ValueError('p must be a leaf')
        # Check if the two trees are the same type
        if not type(self) is type(t1): raise TypeError('Tree types must match')
        
        # Check if the tree added is empty
        if not t1.is_empty():
            root_node = t1._root
            
            # Tree t1's parent link is node p
            root_node._parent = node
            # Put the root of t1 into the children list of p 
            node._children.append(root_node)

            # Increment the size of T
            self._size += len(t1)
            # clean up the tree t1
            t1._root = None
            t1._size = 0
            
class LinkedQueue:
    """Implementation of a Queue using a linked List"""
    
    class _Node:
        """Lightweight non-public class for storing nodes"""
    
        # __slots__ is a tuple that defines the dictionary entries
        __slots__ = ("_element", "_next")
    
    
        def __init__(self, element, next):
            """Constructor of a node object"""
            self._element = element
            self._next = next
    
    
    def __init__(self):
        self._head = None
        self._tail = None
        self._size = 0
        
    def __len__(self):
        return self._size
    
    def is_empty(self):
        return self._size==0
    
    def first(self):
        """Returns the first element in the Queue"""
        if self.is_empty():
            raise Empty("Queue is Empty")
            
        return self._head._element
    
    def dequeue(self):
        """Remove an element from the front of the Queue"""
        
        if self.is_empty():
            raise Empty("Queue is Empty")

        # Grab output element
        output = self._head._element
        
        # re-assign the head of the linked-list
        self._head = self._head._next
        # update the size
        self._size -= 1
        # Check if that was the only element
        if self.is_empty():
            self._tail = None
        return output

    def enqueue(self, e):
        """Adds an element to the back of the Queue"""
        
        # Create a new node
        New_Node = self._Node(e,None)
        
        # if empty it goes as the head=tail
        if self.is_empty():
            self._head = New_Node
        # Normally enqueue to the tail = back of the queue
        else:
            # Update the link to the new tail node
            self._tail._next = New_Node
        # Enqueue'd elements go to the tail = back
        self._tail = New_Node
        # increase the size
        self._size += 1
        
        
# Functions for constructing tree
def my_copy(A):
    """returns a copy of the list A"""
    # Creates a new list
    copy_list = [0]*9
    
    #Loop over input list A to copy it
    for i in range(len(A)):
        copy_list[i] = A[i]
        
    return copy_list



def check_for_win(node):
    """Check if a player has won and stop constructing tree if the game is over.
    Returns True if a player has won, and returns 'a' or 'b' for who won."""
    
    element = node.element()
    e = element[0]
    
    # check the rows
    if e[0] == e[1] == e[2]:
        if e[0] != 0:
            return [True, e[0]]
    if e[3] == e[4] == e[5]:
        if e[3] != 0:
            return [True, e[3]]
    if e[6] == e[7] == e[8]:
        if e[6] != 0:
            return [True, e[6]]
    
    # check the columns
    if e[0] == e[3] == e[6]:
        if e[0] != 0:
            return [True, e[0]]
    if e[1] == e[4] == e[7]:
        if e[1] != 0:
            return [True, e[1]]
    if e[2] == e[5] == e[8]:
        if e[2] != 0:
            return [True, e[2]]
    
    # check the diagonals
    if e[0] == e[4] == e[8]:
        if e[0] != 0:
            return [True, e[0]]
    if e[2] == e[4] == e[6]:
        if e[2] != 0:
            return [True, e[2]]
    
    # otherwise, the game is not over, or it is a draw
    return [False, 0]

def turn(e):
    """Function for checking how many moves have been played.
    Returns 'a' or 'b' depending on which player's turn it is."""
    
    count = 0
    # count how many blank spaces are left
    for i in range(9):
        if e[i] == 0:
            count +=1 
    # if count is even, it is B's turn
    if count % 2 == 0:
        return 'b'
    # if count is odd, it is A's turn
    return 'a'


def play(node):
    """Function for creating the children of a node."""
    # check if the game is over 
    win =  check_for_win(node)
    
    # if the game is not over, create the children
    if win[0] == False:
        for i in range(9):
            old_element = node.element()
            element = old_element[0]
            e = my_copy(element)
            
            # check if the space is empty
            if e[i] == 0:
                # check whose turn it is
                e[i] = turn(e) 
                # add the child with an element that contains the 
                # positional list and the index for the last move
                new_element = [e, i]
                T._add_child(node, [e, i])
                
def printGame(node, x_first):
    """Function to print out the game configuration for a node"""
    # get the element
    e = node.element()
    
    # the first index of the element has the information for 
    # where the players have moved stored in a list
    game = e[0]
    
    # print who the current player is
    if turn(game) == 'b':
        if x_first:
            player = 'X'
        else:
            player = 'O'
    else:
        if x_first:
            player = 'O'
        else:
            player = 'X' 
            
    print('Player = ', player)
    # print the prediction score
    print("Prediction score =", e[2])
    
    # list to store game board with X's and O's
    prnt = [0] * 9
    
    for i in range(9):
        if game[i] == 0:
            prnt[i] = ""
        # if X was the first player, player A is X
        if x_first:
            if game[i] == 'a':
                prnt[i] = 'X'
            if game[i] == 'b':
                prnt[i] = 'O'
        else:
            if game[i] == 'b':
                prnt[i] = 'X'
            if game[i] == 'a':
                prnt[i] = 'O'
    
    print([prnt[0], prnt[1], prnt[2]])
    print([prnt[3], prnt[4], prnt[5]])
    print([prnt[6], prnt[7], prnt[8]])
    
"""Create decision tree"""
T = LinkedTree()

# flag for if X went first 
global x_first
x_first = True

# Add root with empty grid
e = [[0,0,0,0,0,0,0,0,0], None]
T._add_root(e)

# Set node as the root node
node = T.root()

# loop over the possible moves
play(node)

# Get list of the root's children
children = T.children(node)
# create a blank queue
Q = LinkedQueue()

# add the children list to the queue
for i in range(len(children)):
    Q.enqueue(children[i])

# go through Q to create the children of the dequeued element
while len(Q) > 0:
    node = Q.dequeue()
    play(node)
    children = T.children(node)
    # add the children list to the queue
    for i in range(len(children)):
        pos = children[i]
        Q.enqueue(pos)
        
"""Functions for finding the prediction score of a node"""

def scores(node):
    """Returns a prediction score for a node"""
    # start with score and total of 0
    score = 0
    total = 0
    # create a blank queue
    Q = LinkedQueue()
    # check if the node has children
    if T.num_children(node) > 0:
        children = T.children(node)
        # put the children into a queue
        for i in range(T.num_children(node)):
            Q.enqueue(children[i])
    # if the given node is a leaf, calculate score
    else:
        score += check_win_score(node)
        # add one to the total 
        total += 1
    # loop over elements in the queue
    while len(Q) > 0:
        new_node = Q.dequeue()
        # check if the node has children
        if T.num_children(new_node) > 0:
            children = T.children(new_node)
            # put the children into a queue
            for i in range(T.num_children(new_node)):
                Q.enqueue(children[i])
        else:
            # if it does not have children, it is a leaf
            # add the score
            score += check_win_score(new_node)
            # add one to the total 
            total += 1
        
    return (score, total)
        
        
def check_win_score(leaf):
    """Function to check for winner.
    Returns the prediction score for given leaf.
    Returns +1 if first player wins and -1 if second player wins.
    """
    
    score = check_for_win(leaf)
    if score[0] == True:
        if score[1] == 'a':
            return 1
        return -1
    return 0
    
        
def add_score(node):
    """Function to update the element to include the prediction score"""
    # get the score
    score, total = scores(node)

    if total == 0:
        prediction_score = score
    else:
        prediction_score = score / total

    # store the old element
    old_e = node.element()


    # rewrite the element to be the old element plus the prediction score
    node._element = [old_e[0], old_e[1], prediction_score]
    return node._element  

def traverse(T):
    """Traverse the tree and add the prediction scores."""
    node = T.root()
    T._replace(node, add_score(node))

    # create a blank queue
    Q = LinkedQueue()

    # check if the node has children
    if T.num_children(node) > 0:
        children = T.children(node)
        # put the children into a queue
        for i in range(T.num_children(node)):
            Q.enqueue(children[i])
    while len(Q) > 0:
        new_node = Q.dequeue()
        T._replace(new_node, add_score(new_node))
        # check if the new node has children
        if T.num_children(new_node) > 0:
            children = T.children(new_node)
            # put the children into a queue
            for i in range(T.num_children(new_node)):
                Q.enqueue(children[i])
                    
    return T

"""Run the traverse function to add the prediction scores to the tree"""
T = traverse(T)


import tkinter as tk
from tkinter import messagebox
from tkinter import PhotoImage
from random import randrange as rand

# Create the window
window = tk.Tk()

# Create the win/loss/draw counters
global wins
wins = 0
global losses
losses = 0
global draws
draws = 0

# Create the global variable for the position in the tree
global game_pos
game_pos = T.root()



# Create the frames
tictactoe_frame = tk.Frame(window, width=860, height=860, bg='black')
status_frame = tk.Frame(window, width=200, height=860)
# Set the Layout
tictactoe_frame.grid(row=0,column=0)
status_frame.grid(row=0,column=1)



# Create the buttons
# Import the Photos
X_photo = PhotoImage(file="X.png")
O_photo = PhotoImage(file="O.png")
Blank_photo = PhotoImage(file="Blank.png")

# Create a list to store the Labels
game_grid = []
# Loop over the rows
for r in range(3):
    # Create a list for the current row
    row = []
    # Loop over the columns
    for c in range(3):
        # Create a button
        square = tk.Label(tictactoe_frame, image=Blank_photo)
        # Place the button in the grid at position (r,c)
        square.grid(row=r, column=c)
        row.append(square)
    # append the row of buttons to the overall list
    game_grid.append(row)


# Create the status widgets
# Create the hit count label and counter
wins_label = tk.Label(status_frame, text="Wins")
wins_counter = tk.Label(status_frame, text="0")
# Create the miss count label and counter
losses_label = tk.Label(status_frame, text="Losses")
losses_counter = tk.Label(status_frame, text="0")
# Create the draw counter
draws_label = tk.Label(status_frame, text="Draws")
draws_counter = tk.Label(status_frame, text="0")
# Create a spacer widget for the pack layout
spacer = tk.Frame(status_frame)
# Create the start and quit buttons
start_button = tk.Button(status_frame, text="Start")
quit_button = tk.Button(status_frame, text="Quit")

# Construct the right panel
wins_label.pack(side="top", fill=tk.Y, expand=True)
wins_counter.pack(side="top", fill=tk.Y, expand=True)
losses_label.pack(side="top", fill=tk.Y, expand=True)
losses_counter.pack(side="top", fill=tk.Y, expand=True)
draws_label.pack(side="top", fill=tk.Y, expand=True)
draws_counter.pack(side="top", fill=tk.Y, expand=True)
spacer.pack(side="top", fill=tk.BOTH, expand=True,pady=200)
start_button.pack(side="top", fill=tk.BOTH, expand=True,ipady=20)
quit_button.pack(side="top", fill=tk.BOTH, expand=True, ipady=20)


# Define the command Functions
def restart():
    """Function to restart the round"""
    global game_pos
    print('\n\nNew Game')
    # Loop over the game grid
    for r in range(3):
        for c in range(3):
            square = game_grid[r][c]
            # set the image to show the blank square
            square['image'] = Blank_photo
    # update tree position to root
    game_pos = T.root()


def start():
    """Initializes the start of the game"""
    global wins
    global losses
    global draws
    # Clear the scoreboard
    wins=losses=draws=0
    wins_counter['text'] = str(wins)
    losses_counter['text'] = str(losses)
    draws_counter['text'] = str(draws)
    # Call the restart code
    restart()

def quit():
    """Quits the game"""
    window.destroy()

def check_for_moves():
    """A function to check if there are moves left in the game"""
    check = False
    # Loop over the game_grid
    for r in range(3):
        for c in range(3):
            square = game_grid[r][c]
            if square['image'] == Blank_photo.name:
                check = True
    return check

def check_for_draw(player):
    """A function to check if the game is a draw"""
    global draws
    global x_first
    # Check if the game was a draw
    if not check_for_moves():
        # Tell the user that the game is over
        messagebox.showinfo("Information","This game is a draw :/")
        draws+=1
        draws_counter['text'] = str(draws)
        # check who went last and have the other player start the next game
        if player == X_photo.name: 
            x_first = False
        else:
            x_first = True
        # call the restart function
        restart()

def compute_score(player):
    """A function to compute the score of the game"""
    global wins
    global losses
    global x_first
    # Check who won the game
    if player == X_photo.name:         
        # X's wins
        messagebox.showinfo("Information","You Win!!!")
        wins+=1
        wins_counter['text'] = str(wins)
        # if the player wins, O will start the next round
        x_first = False
    else:
        # O's wins
        messagebox.showinfo("Information","You Lose :(")
        losses+=1
        losses_counter['text'] = str(losses)
        # X will start the next round
        x_first = True

def check_winner(player):
    """A function to check if someone won the game or if there is a draw"""
    winner = False
    
    # Check for a winner
    for r in range(3):
        # checking row r
        if game_grid[r][0]['image'] == player and game_grid[r][1]['image'] == player and game_grid[r][2]['image'] == player:
            winner = True
        # check column c
        c = r
        if game_grid[0][c]['image'] == player and game_grid[1][c]['image'] == player and game_grid[2][c]['image'] == player:
            winner = True

    # Check the positive diagonal
    if game_grid[0][0]['image'] == player and game_grid[1][1]['image'] == player and game_grid[2][2]['image'] == player:
        winner = True
        
    # Check the negative diagonal
    if game_grid[0][2]['image'] == player and game_grid[1][1]['image'] == player and game_grid[2][0]['image'] == player:
        winner = True

    # Check if there was a winner
    if winner:
        compute_score(player)
        restart()
    # Check if the game is a draw
    else:
        check_for_draw(player)
        
def two_in_row(children):
    """Function to check if the opponent can win in the next move and if the 
    player will win in the next move if the oppoenent does not block them.
    
    children = list of children of the current game position"""
    
    global game_pos
    global x_first 
    
    # flag for if the player can win in the next move and needs to be blocked
    can_win = False
    
    # loop over all of the children of the curent game position to check if O can win in the next move
    for child in children:
        # get the prediction score
        e = child.element()[2]
        # if X went first, a winning game for O has a score of -1
        if x_first:
            good_o = -1
        # if O went first, a winning game for O has a score of 1
        else:
            good_o = 1
        # if O wins, update the game position
        if e == good_o:
            game_pos = child
            return [True, child.element()[1]]
    # loop over all of the children of the curent game position to check if X can win in their next move
    for child in children:
        # check if X can win on the next move
        kids = T.children(child)
        for kid in kids:
            # get the prediction score
            e = kid.element()[2]
            # if X went first, then a winning game for X has a score of 1
            if x_first:
                good = 1
            # if O went first, a winning game for X has a score of -1
            else:
                good = -1
            # check if X wins 
            if e == good:
                can_win = True
                # get the index for where the winning move can happen
                threat = kid.element()[1]
                break
        if can_win == True:
            break
                
    if can_win == True:
        # update the game position
        # find the child of the current game position that blocks the winning move
        for child in children:
            if child.element()[1] == threat:
                # update the game position
                game_pos = child
        return [True, threat]
    else:
        return [False, None]
        
def best_move():
    """Function for finding the best play"""
    global game_pos
    # get the children of the current game position
    children = T.children(game_pos)
    # list to store the prediction scores of the children
    score_list = []
    # loop over the children
    for child in children:
        # store the children's prediction scores
        score_list.append(child.element()[2])
        
    # if the player went first, the opponent is O and the best move has the lowest prediction score
    if x_first == True:
        best = min(score_list)
    # if the computer went first, the highest prediction score is best
    else:
        best = max(score_list)
    # check if opponent needs to block or can win
    good_move = two_in_row(children)
    if good_move[0] == True:
        # use mod 3 on the integer index of where the opponent needs to go to get the column in the grid
        c = good_move[1] % 3
        # use integer division to get the row
        r = good_move[1] // 3
        return r, c
        
        
    # the opponent will move at the first index with the best score
    move = score_list.index(best)
    # update the game position in the tree
    game_pos = children[move]
    # use mod 3 to get the column
    c = game_pos.element()[1] % 3
    # use integer division to get the row
    r = game_pos.element()[1] // 3

    return r, c

def opponent():
    """Function which runs after the user clicks to move"""
    # get the current game configuration
    global game_pos
    # call the function to determine what the best move is
    r, c = best_move()
    # print the game state
    printGame(game_pos, x_first)
    square = game_grid[r][c]
    # Change the image to O
    square['image'] = O_photo
    # Call the function to check if O has won
    check_winner(O_photo.name)


def find_position(square):
    global game_pos
    """Function to find the tree position corresponding to the player's move"""
    # find the correct row and column combination for the square the player clicked on 
    for i in range(3):
        r = i
        for i in range(3):
            c = i
            if square == game_grid[r][c]:
                # store the matching row and column numbers
                grid_space = [r, c]
                
    # determine which integer 0-8 matches that row and column
    for i in range(9):
        r1 = i // 3
        c1 = i % 3
        if grid_space == [r1, c1]:
            grid_index = i
            
    # get old game position's children
    children = T.children(game_pos)
    # see what child matches the move that was made
    for child in children:
        if grid_index == child.element()[1]:
            # update the game position
            game_pos = child

    
def click(event):
    """Function which runs when a square is clicked on"""
    global game_pos
    # Determine which button was hit, i.e. the widget the event is from
    square = event.widget

    if square['image'] == Blank_photo.name:
        # Change the image to X
        square['image'] = X_photo
        # update the game position
        find_position(square)
        # print the current game state
        printGame(game_pos, x_first)
        # Check to see if X won the game
        check_winner(X_photo.name)
        # Call the opponent function
        opponent()
    else:
        # Tell the user that the position is already taken
        messagebox.showinfo("Information","This position is not a valid move")


# Set the command actions
# Loop over the game_grid and set the command for each
for r in range(3):
    for c in range(3):
        square = game_grid[r][c]
        square.bind("<ButtonPress-1>",click)
# Set the command for the quit button
start_button['command'] = start
# Set the command for the quit button
quit_button['command'] = quit


# Start the GUI event loop
window.attributes('-topmost',True)
window.mainloop()
